In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras import backend as K
from keras.models import load_model
import cv2
import numpy as np

In [2]:
# Hyper parameters
batch_size = 128
num_classes = 10
epochs = 10

In [3]:
# Split test and train
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape image to (28x28)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

input_shape = (28, 28, 1)

# Conversion of class vectors to matrices of binary class 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Convert to float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize by dividing by 255 to keep the data in the range of 0-1
x_train /= 255
x_test /= 255

In [4]:
# Initialize sequential, layer-based model
model = Sequential()

# 1st Convolutional layer: Applies 32 convolution filters of size 3x3 with ReLU activation to the input image.
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# 2nd Convolutional layer: Applies 64 convolution filters of size 3x3 with ReLU activation.
model.add(Conv2D(64, (3, 3), activation='relu'))

# MaxPooling layer: Reduces the dimensionality by taking the maximum value in a 2x2 pool.
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer: Randomly sets a fraction of input units to 0 during training to prevent overfitting.
model.add(Dropout(0.25))

# Flatten layer: Flattens the input, converting the 2D matrix into a 1D vector.
model.add(Flatten())

# Fully connected (Dense) layer: Applies 256 units with ReLU activation.
model.add(Dense(256, activation='relu'))

# Dropout layer: Randomly sets a fraction of input units to 0 during training to prevent overfitting.
model.add(Dropout(0.5))

# Fully connected (Dense) layer: Applies 'num_classes' units with softmax activation to output class probabilities.
model.add(Dense(num_classes, activation='softmax'))

# Compile the model: Specify the loss function, optimizer, and metrics for evaluation.
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

In [5]:
# Train the model
hist = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test)
)
print("The model has been successfully trained.")

Epoch 1/10
469/469 [==============================] - 8s 8ms/step - loss: 2.2523 - accuracy: 0.2109 - val_loss: 2.1778 - val_accuracy: 0.4921
Epoch 2/10
469/469 [==============================] - 3s 7ms/step - loss: 2.1318 - accuracy: 0.3839 - val_loss: 2.0283 - val_accuracy: 0.6369
Epoch 3/10
469/469 [==============================] - 3s 7ms/step - loss: 1.9756 - accuracy: 0.5081 - val_loss: 1.8307 - val_accuracy: 0.7101
Epoch 4/10
469/469 [==============================] - 3s 7ms/step - loss: 1.7746 - accuracy: 0.5853 - val_loss: 1.5835 - val_accuracy: 0.7513
Epoch 5/10
469/469 [==============================] - 3s 7ms/step - loss: 1.5451 - accuracy: 0.6347 - val_loss: 1.3197 - val_accuracy: 0.7784
Epoch 6/10
469/469 [==============================] - 3s 7ms/step - loss: 1.3282 - accuracy: 0.6649 - val_loss: 1.0877 - val_accuracy: 0.8035
Epoch 7/10
469/469 [==============================] - 3s 7ms/step - loss: 1.1469 - accuracy: 0.6960 - val_loss: 0.9102 - val_accuracy: 0.8230
Epoch 

In [6]:
# Save the model
model.save('mnist.h5')
print("Saving the model as mnist.h5")

Saving the model as mnist.h5


In [7]:
# Evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6223423480987549
Test accuracy: 0.8557999730110168


In [9]:
# Load the model for prediction
model = load_model('mnist.h5')

def preprocess_frame(frame):
    """Preprocess the camera frame for prediction."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (28, 28))
    reshaped = resized.reshape(1, 28, 28, 1).astype('float32')
    normalized = reshaped / 255.0
    return normalized

# Initialize the camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    processed_frame = preprocess_frame(frame)

    # Make prediction
    prediction = model.predict(processed_frame)
    predicted_class = np.argmax(prediction)

    # Display the prediction on the frame
    cv2.putText(frame, f'Predicted: {predicted_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow('Live Camera', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step
